In [1]:
import linguamind.linalg as la
import linguamind.nn as nn
import linguamind.nlp as nlp

from collections import Counter
import random

path ='/Users/amberedmundson/Laboratory/datasets/harryPotter/harryPotterAsciiCleaned.txt'
# path = '/Users/amberedmundson/Laboratory/clones/word2vec/text.txt'

v = nlp.Vocab()
t = nlp.Text(path,v)

v2 = nlp.Vocab(t.vocab)
v2.sort(10)

In [39]:
t.ChangeVocab(v2)
v2.InitUnigramTable()

vocab_size = v2.size
dim = 32

seed = la.Seed(1)

syn0 = nn.SparseLinearInput(vocab_size,dim)
syn0.weights.uniform(seed)
syn0.weights -= 0.5
syn0.weights /= dim

syn2 = nn.Linear(dim,dim)
syn2.weights.uniform(seed)
syn2.weights -= 0.5
syn2.weights /= dim

syn1neg = nn.SparseLinearOutput(dim,vocab_size)
syn1neg.weights.zero()

layers = list()
layers.append(syn0)
layers.append(syn1neg)
layers.append(nn.Sigmoid(vocab_size))

mlp = nn.Sequential(layers)
criterion = nn.MSECriterion()
optim = nn.StochasticGradient(mlp,criterion)

In [40]:
words = 0
for sent in t.sentences:
    for word in sent:
        words+=1

In [41]:
cbow = nn.CBOW(t.sentences,v2,5,5)

In [10]:
%%timeit
optim.train(cbow,0.01,1,1)

1 loop, best of 3: 58.1 s per loop


In [11]:
%%timeit
optim.train(cbow,0.01,1,4)

1 loop, best of 3: 28.9 s per loop


In [44]:
%%timeit
optim.train(cbow,0.01,3,4)

1 loop, best of 3: 3.89 s per loop


In [45]:
words

1381594

In [43]:
import numpy as np

word2vec = {}
vocab = list()
for i in range(v2.size):
    l2 = np.sqrt(syn0.weights[i].dot(syn0.weights[i]))
    v = syn0.weights[i]
    v /= l2
    vocab.append(v2.getTermAtIndex(i).letters)
    word2vec[v2.getTermAtIndex(i).letters] = syn0.weights[i]
    
sim = Counter()
for word in vocab:
    sim[word] = word2vec['ron'].dot(word2vec[word])
sim.most_common()

[('ron', 1.0000001192092896),
 ('hermione', 0.9049298167228699),
 ('ginny', 0.868349015712738),
 ('lupin', 0.8383578062057495),
 ('neville', 0.8333842754364014),
 ('hagrid', 0.774434506893158),
 ('luna', 0.7707253694534302),
 ('malfoy', 0.7621385455131531),
 ('bill', 0.7523704767227173),
 ('umbridge', 0.748776376247406),
 ('fred', 0.7360839247703552),
 ('george', 0.7356505393981934),
 ('fleur', 0.732907235622406),
 ('dean', 0.7270064353942871),
 ('snape', 0.7212597131729126),
 ('tonks', 0.7040488719940186),
 ('parvati', 0.7037661075592041),
 ('fudge', 0.6999493837356567),
 ('quickly', 0.6961274743080139),
 ('dumbledore', 0.6951295733451843),
 ('krum', 0.6921409964561462),
 ('bagman', 0.6899760365486145),
 ('maxime', 0.6862226128578186),
 ('goyle', 0.6847946643829346),
 ('seamus', 0.6790963411331177),
 ('she', 0.6781429052352905),
 ('crabbe', 0.6774758100509644),
 ('lavender', 0.6746861934661865),
 ('sirius', 0.6736510396003723),
 ('cho', 0.67322838306427),
 ('moody', 0.6706942915916443

# Forward Prop Modification for Windowing Layers

after you have confirmed that you are doing everything that word2vec does (exactly the same!!!)... such that you duplicate results, you should try this highly experimental thing

For CBOW, you should be able to tweak the output embedding based on how the inputs change and the previous gradient instead of regenerating it every time. If your window size is 5, this should give you close to a 5 * 2 = 10x speedup for the input window generation (which is where most of the parameters are going anyway)